# Machine Translation

In [ ]:
!nvidia-smi

Sun May  9 14:33:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python -m spacy download de
!python -m spacy download en

     |████████████████████████████████| 14.9MB 11.7MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907057 sha256=2a119121b9af0e44f0526eeab5a3cad49495f47561c80255808bfe695ea612e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ndqr_90p/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
!pip install --upgrade torchtext==0.4.0

     |████████████████████████████████| 61kB 7.5MB/s 
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [ ]:
SEED = 45

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.determinitic = True

In [ ]:
spacy_en = spacy.load('en')
spacy_de = spacy.load('de')

In [ ]:
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)][::-1]

In [ ]:
SRC = Field(tokenize=tokenize_de,
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)

TRG = Field(tokenize=tokenize_en,
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'),
                                                    fields=(SRC,TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 1.12MB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 274kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 268kB/s]


In [ ]:
vars(train_data[0])

{'src': ['.',
  'büsche',
  'vieler',
  'nähe',
  'der',
  'in',
  'freien',
  'im',
  'sind',
  'männer',
  'weiße',
  'junge',
  'zwei'],
 'trg': ['two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.']}

In [ ]:
print("Length of train data: ",len(train_data))
print("Length of valid data: ",len(valid_data))
print("Length of test data: ",len(test_data))

Length of train data:  29000
Length of valid data:  1014
Length of test data:  1000


In [ ]:
SRC.build_vocab(train_data)

In [ ]:
type(SRC.vocab)

torchtext.vocab.Vocab

In [ ]:
vars(SRC.vocab)

{'freqs': Counter({'.': 28821,
          'büsche': 5,
          'vieler': 2,
          'nähe': 383,
          'der': 4989,
          'in': 11893,
          'freien': 475,
          'im': 3107,
          'sind': 490,
          'männer': 1662,
          'weiße': 146,
          'junge': 2068,
          'zwei': 3873,
          'antriebsradsystem': 1,
          'ein': 18850,
          'bedienen': 9,
          'schutzhelmen': 33,
          'mit': 8843,
          'mehrere': 580,
          'holz': 61,
          'aus': 910,
          'spielhaus': 2,
          'klettert': 171,
          'mädchen': 2121,
          'kleines': 772,
          'fenster': 148,
          'putzt': 43,
          'und': 8925,
          'leiter': 56,
          'einer': 6765,
          'auf': 8745,
          'steht': 1778,
          'hemd': 1202,
          'blauen': 992,
          'einem': 13711,
          'mann': 7805,
          'zu': 1909,
          'essen': 377,
          'bereiten': 79,
          'herd': 17,
          '

In [ ]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [ ]:
print("Source vocabulary length: ",len(SRC.vocab))
print("Target vocabulary length: ",len(TRG.vocab))

Source vocabulary length:  7855
Target vocabulary length:  5893


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
BATCH_SIZE=128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device
)

In [ ]:
vars(train_iterator)

{'_iterations_this_epoch': 0,
 '_random_state_this_epoch': None,
 '_restored_from_state': False,
 'batch_size': 128,
 'batch_size_fn': None,
 'dataset': <torchtext.datasets.translation.Multi30k at 0x7f85f3c4e8d0>,
 'device': 'cuda',
 'iterations': 0,
 'random_shuffler': <torchtext.data.utils.RandomShuffler at 0x7f853f09f050>,
 'repeat': False,
 'shuffle': True,
 'sort': False,
 'sort_key': <function torchtext.datasets.translation.TranslationDataset.sort_key>,
 'sort_within_batch': False,
 'train': True}

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hidden_dim == decoder.hidden_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.num_layers == decoder.num_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio=0.5):

        hidden, cell = self.encoder(src)
        input = trg[0,:]

        trg_len = trg.shape[0]
        batch_size = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device) # one-hot vector

        for i in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[i] = output

            teacher_forcing = random.random() < teacher_forcing_ratio
            input = trg[i] if teacher_forcing else output.argmax(1)

        return outputs


In [ ]:
class Encoder(nn.Module):
    def __init__(self, dropout, input_dim, embedding_dim, hidden_dim, num_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=dropout)


    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        _, (hidden, cell) = self.rnn(embedded)

        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, dropout, embedding_dim, hidden_dim, num_layers, output_dim):
        super().__init__()

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded,(hidden, cell))
        prediction = self.fc(output.squeeze(0))

        return prediction, hidden, cell

In [ ]:
INPUT_DIM = len(SRC.vocab)
EMBEDDING_DIM = 256
HIDDEN_DIM = 512
DROPOUT = 0.2
NUM_LAYERS = 2
OUTPUT_DIM = len(TRG.vocab)

Encoder = Encoder(DROPOUT, INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, NUM_LAYERS)
Decoder = Decoder(DROPOUT, EMBEDDING_DIM, HIDDEN_DIM, NUM_LAYERS, OUTPUT_DIM)

model = Seq2Seq(Encoder, Decoder, device).to(device)

In [ ]:
def init_weights(m):
    for _,param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.2)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.2)
    (fc): Linear(in_features=512, out_features=5893, bias=True)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('Number of parameters: ', count_parameters(model))

Number of parameters:  13899013


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
TRG_PAD_IDX

1

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index= TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    epoch_loss = 0
    model.train()

    for batch in iterator:
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
    
    return epoch_loss/len(iterator)

In [ ]:
def evaluate(model, iterator, optimizer, criterion):
    epoch_loss = 0
    model.eval()

    with torch.no_grad():
        for batch in iterator:
            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0)

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
        return epoch_loss/len(iterator)

In [ ]:
def epoch_time(start, end):
    time = end - start
    mins = int(time/60)
    secs = int(time - (mins*60))
    return mins, secs

In [ ]:
EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(EPOCHS):

    start = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, optimizer, criterion)

    end = time.time()

    mins, secs = epoch_time(start, end)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'machine-translation-de-en.pt')

    print(f'Epoch: {epoch} | Time: {mins}m {secs}s')
    print(f'Train loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f} ')
    print(f'Valid loss: {valid_loss:.3f} | Valid PPL: {math.exp(valid_loss):7.3f} ')

Epoch: 0 | Time: 0m 28s
Train loss: 5.035 | Train PPL: 153.672 
Valid loss: 5.020 | Valid PPL: 151.419 
Epoch: 1 | Time: 0m 28s
Train loss: 4.431 | Train PPL:  83.977 
Valid loss: 4.773 | Valid PPL: 118.285 
Epoch: 2 | Time: 0m 28s
Train loss: 4.090 | Train PPL:  59.758 
Valid loss: 4.596 | Valid PPL:  99.081 
Epoch: 3 | Time: 0m 28s
Train loss: 3.875 | Train PPL:  48.190 
Valid loss: 4.465 | Valid PPL:  86.951 
Epoch: 4 | Time: 0m 28s
Train loss: 3.693 | Train PPL:  40.164 
Valid loss: 4.328 | Valid PPL:  75.767 
Epoch: 5 | Time: 0m 28s
Train loss: 3.516 | Train PPL:  33.655 
Valid loss: 4.144 | Valid PPL:  63.033 
Epoch: 6 | Time: 0m 28s
Train loss: 3.370 | Train PPL:  29.084 
Valid loss: 4.110 | Valid PPL:  60.942 
Epoch: 7 | Time: 0m 28s
Train loss: 3.195 | Train PPL:  24.411 
Valid loss: 3.958 | Valid PPL:  52.343 
Epoch: 8 | Time: 0m 28s
Train loss: 3.046 | Train PPL:  21.037 
Valid loss: 3.902 | Valid PPL:  49.483 
Epoch: 9 | Time: 0m 28s
Train loss: 2.917 | Train PPL:  18.488 


In [ ]:
model(input)